In [2]:
import optuna
import pandas as pd
import numpy as np
import xgboost as xgb
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import roc_auc_score

In [3]:
insp = pd.read_csv("inspections.csv")
network = pd.read_csv("network.csv")
samplesub = pd.read_csv("sample_submission.csv")

In [4]:
insp

,PipeId,MaintenanceId,InspectionYear,InspectionDate,MonthsLastRev,Severity,Incidence
0,189311802,ZRV-00001972,2010,2010-10-01,24,4,0
1,188889914,ZRV-00001406,2010,2010-10-01,22,4,0
2,188889907,ZRV-00001406,2010,2010-10-01,22,4,0
3,188673355,ZRV-00001406,2010,2010-10-01,22,4,0
4,188558922,ZRV-00001406,2010,2010-10-01,22,4,0
...,...,...,...,...,...,...,...
6345339,280560178,ZRV-00002112,2021,2020-12-31,23,4,0
6345340,274990683,ZRV-00002112,2021,2020-12-31,23,4,0
6345341,280560172,ZRV-00002112,2021,2020-12-31,23,4,0
6345342,274990921,ZRV-00002112,2021,2020-12-31,23,4,0


In [5]:
latest_year_index = insp.groupby('PipeId')['InspectionYear'].idxmax()
filtered_df = insp.loc[latest_year_index]

In [6]:
filtered_df

In [8]:
network = network.merge(filtered_df, how='left', on='PipeId')

In [9]:
network = network.drop(columns=["MaintenanceId","InspectionDate"])

In [10]:
network

,PipeId,Province,Town,YearBuilt,Material,GasType,Diameter,Length,Pressure,NumConnections,NumConnectionsUnder,BoolBridle,InspectionYear,MonthsLastRev,Severity,Incidence
0,489616,Ciudad Real,Torralba de Calatrava,2012,PE,Gas natural,63.0,7.792,0.4,0,0,False,2020.0,24.0,4.0,0.0
1,489645,Ciudad Real,Torralba de Calatrava,2012,PE,Gas natural,90.0,2.080,0.4,0,0,False,2020.0,24.0,4.0,0.0
2,489646,Ciudad Real,Torralba de Calatrava,2012,PE,Gas natural,90.0,111.173,0.4,3,0,False,2020.0,24.0,4.0,0.0
3,519950,Ciudad Real,Torralba de Calatrava,2012,PE,Gas natural,200.0,14.304,0.4,0,0,False,2020.0,24.0,4.0,0.0
4,1332439,Ciudad Real,Torralba de Calatrava,2012,PE,Gas natural,160.0,22.669,0.4,1,0,False,2020.0,24.0,4.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1446534,416536087,La Rioja,Baños de Rio Tobia,2011,AO,Gas natural,50.8,0.454,45.0,0,0,False,2013.0,0.0,4.0,0.0
1446535,416536416,La Rioja,Baños de Rio Tobia,2011,AO,Gas natural,101.6,0.478,45.0,0,0,False,2013.0,0.0,4.0,0.0
1446536,416536438,La Rioja,Baños de Rio Tobia,2011,AO,Gas natural,101.6,0.485,45.0,0,0,False,2013.0,0.0,4.0,0.0
1446537,416536460,La Rioja,Baños de Rio Tobia,2011,AO,Gas natural,50.8,0.210,45.0,0,0,False,2013.0,0.0,4.0,0.0


In [11]:
filtered2 = insp.drop(filtered_df.index)
latest_year_index_2 = filtered2.groupby('PipeId')['InspectionYear'].idxmax()
filtered2= filtered2.loc[latest_year_index_2]
filtered2 = filtered2[["PipeId","Severity"]]
filtered2.rename(columns = {"Severity":"PreviousInsSeverity"}, inplace = True)
filtered2

In [16]:
network = network.merge(filtered2, how='left', on='PipeId')

In [17]:
network = network.drop(columns="Severity",axis=1)

In [18]:
network["Volume"] = ((3.14*(network["Diameter"]/2000))**2*network["Length"])


In [19]:
network["Age"]

,PipeId,Province,Town,YearBuilt,Material,GasType,Diameter,Length,Pressure,NumConnections,NumConnectionsUnder,BoolBridle,InspectionYear,MonthsLastRev,Incidence,PreviousInsSeverity,Volume
0,489616,Ciudad Real,Torralba de Calatrava,2012,PE,Gas natural,63.0,7.792,0.4,0,0,False,2020.0,24.0,0.0,4.0,0.076231
1,489645,Ciudad Real,Torralba de Calatrava,2012,PE,Gas natural,90.0,2.080,0.4,0,0,False,2020.0,24.0,0.0,4.0,0.041529
2,489646,Ciudad Real,Torralba de Calatrava,2012,PE,Gas natural,90.0,111.173,0.4,3,0,False,2020.0,24.0,0.0,4.0,2.219646
3,519950,Ciudad Real,Torralba de Calatrava,2012,PE,Gas natural,200.0,14.304,0.4,0,0,False,2020.0,24.0,0.0,4.0,1.410317
4,1332439,Ciudad Real,Torralba de Calatrava,2012,PE,Gas natural,160.0,22.669,0.4,1,0,False,2020.0,24.0,0.0,4.0,1.430447
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1446534,416536087,La Rioja,Baños de Rio Tobia,2011,AO,Gas natural,50.8,0.454,45.0,0,0,False,2013.0,0.0,0.0,NaN,0.002888
1446535,416536416,La Rioja,Baños de Rio Tobia,2011,AO,Gas natural,101.6,0.478,45.0,0,0,False,2013.0,0.0,0.0,NaN,0.012162
1446536,416536438,La Rioja,Baños de Rio Tobia,2011,AO,Gas natural,101.6,0.485,45.0,0,0,False,2013.0,0.0,0.0,NaN,0.012340
1446537,416536460,La Rioja,Baños de Rio Tobia,2011,AO,Gas natural,50.8,0.210,45.0,0,0,False,2013.0,0.0,0.0,NaN,0.001336


In [20]:
network_reworked = network

In [21]:
network_reworked

,PipeId,Province,Town,YearBuilt,Material,GasType,Diameter,Length,Pressure,NumConnections,NumConnectionsUnder,BoolBridle,InspectionYear,MonthsLastRev,Incidence,PreviousInsSeverity,Volume
0,489616,Ciudad Real,Torralba de Calatrava,2012,PE,Gas natural,63.0,7.792,0.4,0,0,False,2020.0,24.0,0.0,4.0,0.076231
1,489645,Ciudad Real,Torralba de Calatrava,2012,PE,Gas natural,90.0,2.080,0.4,0,0,False,2020.0,24.0,0.0,4.0,0.041529
2,489646,Ciudad Real,Torralba de Calatrava,2012,PE,Gas natural,90.0,111.173,0.4,3,0,False,2020.0,24.0,0.0,4.0,2.219646
3,519950,Ciudad Real,Torralba de Calatrava,2012,PE,Gas natural,200.0,14.304,0.4,0,0,False,2020.0,24.0,0.0,4.0,1.410317
4,1332439,Ciudad Real,Torralba de Calatrava,2012,PE,Gas natural,160.0,22.669,0.4,1,0,False,2020.0,24.0,0.0,4.0,1.430447
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1446534,416536087,La Rioja,Baños de Rio Tobia,2011,AO,Gas natural,50.8,0.454,45.0,0,0,False,2013.0,0.0,0.0,NaN,0.002888
1446535,416536416,La Rioja,Baños de Rio Tobia,2011,AO,Gas natural,101.6,0.478,45.0,0,0,False,2013.0,0.0,0.0,NaN,0.012162
1446536,416536438,La Rioja,Baños de Rio Tobia,2011,AO,Gas natural,101.6,0.485,45.0,0,0,False,2013.0,0.0,0.0,NaN,0.012340
1446537,416536460,La Rioja,Baños de Rio Tobia,2011,AO,Gas natural,50.8,0.210,45.0,0,0,False,2013.0,0.0,0.0,NaN,0.001336


In [25]:
insp["NumberOfInspections"] = (insp["PipeId"].groupby(insp['PipeId']).count())

In [26]:
NoI = pd.DataFrame(insp["PipeId"].groupby(insp['PipeId']).count())

In [27]:
NoI["NoI"] = NoI["PipeId"]

In [28]:
NoI = NoI.drop("PipeId",axis=1)

In [29]:
NoI

,NoI
PipeId,
489616,5
489645,5
489646,5
489780,5
489790,5
...,...
451195430,4
451195580,4
451995254,3


In [30]:
network_reworked_sorted = network_reworked_sorted.merge(NoI, how = "left", on="PipeId")

In [31]:
network_reworked_sorted

,PipeId,Province,Town,YearBuilt,Material,GasType,Diameter,Length,Pressure,NumConnections,NumConnectionsUnder,BoolBridle,InspectionYear,MonthsLastRev,Incidence,PreviousInsSeverity,Volume,NoI
0,15663411,Madrid,Madrid,1997,PE,Gas natural,160.0,25.039,0.025,2,0,False,2021.0,21.0,0.0,4.0,1.579997,6.0
1,190196719,Valencia,Valencia,2002,PE,Gas natural,110.0,6.765,0.100,0,0,False,2021.0,23.0,0.0,4.0,0.201768,6.0
2,190225259,Valencia,Valencia,2002,PE,Gas natural,110.0,0.988,0.100,0,0,False,2021.0,23.0,0.0,4.0,0.029467,5.0
3,190225252,Valencia,Valencia,2002,PE,Gas natural,110.0,42.257,0.100,3,0,False,2021.0,23.0,0.0,4.0,1.260327,6.0
4,190224497,Valencia,Valencia,2002,PE,Gas natural,110.0,52.974,0.100,4,0,False,2021.0,23.0,0.0,4.0,1.579965,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1446534,398048735,La Rioja,Ezcaray,2011,AO,Gas natural,50.8,0.350,16.000,0,0,False,NaN,NaN,0.0,NaN,0.002226,NaN
1446535,398048765,La Rioja,Ezcaray,2011,AO,Gas natural,101.6,0.775,16.000,0,0,False,NaN,NaN,0.0,NaN,0.019719,NaN
1446536,416067866,La Rioja,Cirueña,2011,AO,Gas natural,50.8,0.758,45.000,0,0,False,NaN,NaN,0.0,NaN,0.004822,NaN
1446537,416223449,La Rioja,Cirueña,2011,AO,Gas natural,50.8,0.460,45.000,0,0,False,NaN,NaN,0.0,NaN,0.002926,NaN


In [32]:
samplesub = samplesub.drop(columns="Incidence",axis=1)

In [33]:
samplesub = samplesub.merge(network_reworked_sorted, how = "left", on= "PipeId")

In [34]:
samplesub = samplesub.drop("Incidence", axis=1)

In [35]:
samplesub.InspectionYear.value_counts()

2019.0    634458
2020.0    228158
2017.0     18956
2018.0      5159
2015.0      1970
2016.0       924
2013.0       835
2014.0       289
2011.0       162
2012.0       133
2010.0         4
Name: InspectionYear, dtype: int64

In [39]:
network_reworked_sorted

,PipeId,Province,Town,YearBuilt,Material,GasType,Diameter,Length,Pressure,NumConnections,NumConnectionsUnder,BoolBridle,InspectionYear,MonthsLastRev,Incidence,PreviousInsSeverity,Volume,NoI
0,15663411,Madrid,Madrid,1997,PE,Gas natural,160.0,25.039,0.025,2,0,False,2021.0,21.0,0.0,4.0,1.579997,6.0
1,190196719,Valencia,Valencia,2002,PE,Gas natural,110.0,6.765,0.100,0,0,False,2021.0,23.0,0.0,4.0,0.201768,6.0
2,190225259,Valencia,Valencia,2002,PE,Gas natural,110.0,0.988,0.100,0,0,False,2021.0,23.0,0.0,4.0,0.029467,5.0
3,190225252,Valencia,Valencia,2002,PE,Gas natural,110.0,42.257,0.100,3,0,False,2021.0,23.0,0.0,4.0,1.260327,6.0
4,190224497,Valencia,Valencia,2002,PE,Gas natural,110.0,52.974,0.100,4,0,False,2021.0,23.0,0.0,4.0,1.579965,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1446534,398048735,La Rioja,Ezcaray,2011,AO,Gas natural,50.8,0.350,16.000,0,0,False,NaN,NaN,0.0,NaN,0.002226,NaN
1446535,398048765,La Rioja,Ezcaray,2011,AO,Gas natural,101.6,0.775,16.000,0,0,False,NaN,NaN,0.0,NaN,0.019719,NaN
1446536,416067866,La Rioja,Cirueña,2011,AO,Gas natural,50.8,0.758,45.000,0,0,False,NaN,NaN,0.0,NaN,0.004822,NaN
1446537,416223449,La Rioja,Cirueña,2011,AO,Gas natural,50.8,0.460,45.000,0,0,False,NaN,NaN,0.0,NaN,0.002926,NaN


In [40]:
#samplesub.to_csv("samplesub_mod.csv")
#network_reworked_sorted.to_csv("network_mod.csv")